In [1]:
import sys
import os
import re
import numpy as np
import pandas as pd
import time
import psycopg2
import requests
from bs4 import BeautifulSoup

from convertPDF.webScrape.pdf2xml import *
from convertPDF.webScrape.xml2txt import *

from convertPDF.driver import parseFullDay

from convertPDF.infoFns.genInfoFns import parseGenInfo
from convertPDF.infoFns.horseInfoFns import parseHorseInfo
from convertPDF.infoFns.timesInfoFns import parseTimeInfo
from convertPDF.infoFns.betInfoFns import parseBetInfo
from convertPDF.infoFns.runlineInfoFns import parseRunlineInfo
from convertPDF.infoFns.endInfoFns import parseEndInfo

from populateDB import generateEntries, populateDB

from convertPDF.infoFns.regexPatterns import *

from convertPDF.webScrape.htmlMgmt import *
from convertPDF.webScrape.saveTrackUrlsFromFiles import saveRaceUrlsFromFiles

from dataClean.cleanFns import racesClean, horsesClean

In [2]:
horsesdf = pd.read_csv('./../outputs/horses.csv', dtype='string')
racesdf = pd.read_csv('./../outputs/races.csv', dtype='string')
horsesdf = horsesClean(horsesdf)
racesdf = racesClean(racesdf)

In [4]:
horsesdf.to_csv('./../../horseAnalysis/data/horses.csv', index=False)
racesdf.to_csv('./../../horseAnalysis/data/races.csv', index=False)

In [2]:
start = time.time()
entries = generateEntries('./../charts/txts')
end = time.time()

end - start

Converting 6421 files
0
300
Error in parseBetLine on line:  $1.00 Daily Double 5-2 7.70 3,587

600
900
1200
Error in parseAdditionalBetLines on line:  $0.20 Superfecta 11-4-3-2 7,744 5,808

Error in parseBetLine on line:  $1.00 Daily Double 5-11 9.00 3,947

1500
Match error in parseStart on line:  Off at: : Start: Good for all except

Match error in parseFractionalTimes on line:  Final Time:

Match error in parseStart on line:  Off at: : Start: Good for all except

Match error in parseFractionalTimes on line:  Final Time:

1800
Match error in parseStart on line:  Off at: : Start: Good for all except

Match error in parseFractionalTimes on line:  Final Time:

Match error in parseStart on line:  Off at: : Start: Good for all except

Match error in parseFractionalTimes on line:  Final Time:

Match error in parseStart on line:  Off at: : Start: Good for all except

Match error in parseFractionalTimes on line:  Final Time:

Match error in parseStart on line:  Off at: : Start: Good for all e

192.39600467681885

In [3]:
conn = psycopg2.connect(
    host = "localhost",
    database = "horses",
    user = "karisch",
    password = "cocacola",
    port = 5432
)

In [4]:
populateDB(conn, entries)

UniqueViolation: duplicate key value violates unique constraint "horses_pkey"
DETAIL:  Key (track, date, race, "horseProgram")=(CMR, 6/26/2021, 5, ERROR) already exists.


In [5]:
with conn.cursor() as cur:
    cur.execute("SELECT * FROM main.races;")
    races = cur.fetchall()
    cur.execute("SELECT * FROM main.horses;")
    horses = cur.fetchall()

In [10]:
raceCols = ("track", "date","race","stakes","distanceStr","surface","weather","conditions",            "startTime","startNote","segment1","segment2","segment3","segment4","segment5","segments","fracTime1","fracTime2","fracTime3","fracTime4","fracTime5","finalTime","runup","wpsPool","firstPlaceWin","firstPlacePlace","firstPlaceShow","secondPlacePlace","secondPlaceShow","thirdPlaceShow","exactaBuyin","exactaFinish","exactaPayout","exactaPool","trifectaBuyin","trifectaFinish","trifectaPayout","trifectaPool","superfectaBuyin","superfectaFinish","superfectaPayout","superfectaPool","quinellaBuyin","quinellaFinish","quinellaPayout","quinellaPool")

In [7]:
horseCols = ("track","date","race","horseProgram","horseName","lastRaceDay","lastRaceMonth","lastRaceYear","lastRaceTrack","jockey","weight","ME","placePP","placeSeg1","placeSeg2","placeSeg3","placeSeg4","placeSeg5","placeSeg6","odds","comments","lastRaceNum","lastRacePlace","lengthsSeg1","lengthsSeg2","lengthsSeg3","lengthsSeg4","lengthsSeg5","lengthsSeg6","rlLengthsSeg1","rlLengthsSeg2","rlLengthsSeg3","rlLengthsSeg4","rlLengthsSeg5","rlLengthsSeg6","rlPlaceSeg1","rlPlaceSeg2","rlPlaceSeg3","rlPlaceSeg4","rlPlaceSeg5","rlPlaceSeg6","trainer","owner")

In [11]:
racesdf = pd.DataFrame(races, columns=raceCols, dtype='string')
horsesdf = pd.DataFrame(horses, columns=horseCols, dtype='string')

In [12]:
racesdf.to_csv('./../outputs/races.csv', index=False)
horsesdf.to_csv('./../outputs/horses.csv', index=False)

In [2]:
pdf2xml('./../charts/pdfs/2021/', './../charts/xmls/')

Converting  2296 files


In [3]:
bulkXml2Txt('./../charts/xmls/', './../charts/txts/')

Converting 6422 files
Progress: 14.0%
